In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = './data/'

In [3]:
currency_pair = 'Usd_Chf'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [5]:
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume
0,2021-04-01 06:00:00,0.94570,0.94592,0.94570,0.94589,0.94586,0.94607,0.94586,0.94603,0.94578,0.94600,0.94578,0.94596,12
1,2021-04-01 06:05:00,0.94593,0.94596,0.94551,0.94559,0.94607,0.94610,0.94565,0.94575,0.94600,0.94603,0.94558,0.94567,42
2,2021-04-01 06:10:00,0.94564,0.94575,0.94551,0.94551,0.94577,0.94590,0.94566,0.94566,0.94570,0.94582,0.94558,0.94558,34
3,2021-04-01 06:15:00,0.94554,0.94572,0.94534,0.94537,0.94569,0.94587,0.94548,0.94552,0.94562,0.94579,0.94541,0.94544,40
4,2021-04-01 06:20:00,0.94534,0.94541,0.94476,0.94490,0.94549,0.94556,0.94491,0.94506,0.94542,0.94548,0.94484,0.94498,35


In [6]:
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume
73010,2022-04-01 05:35:00,0.92364,0.92399,0.92364,0.92393,0.92386,0.92415,0.92386,0.92411,0.92375,0.92407,0.92375,0.92402,117
73011,2022-04-01 05:40:00,0.92393,0.92394,0.92349,0.92352,0.92409,0.92411,0.92364,0.92364,0.92401,0.92402,0.92358,0.92358,194
73012,2022-04-01 05:45:00,0.92351,0.92359,0.92348,0.92357,0.92365,0.92376,0.92361,0.92375,0.92358,0.92368,0.92355,0.92366,98
73013,2022-04-01 05:50:00,0.92359,0.92364,0.92348,0.92349,0.92375,0.92379,0.92363,0.92365,0.92367,0.92370,0.92356,0.92357,109
73014,2022-04-01 05:55:00,0.92353,0.92371,0.92348,0.92366,0.92369,0.92386,0.92363,0.92382,0.92361,0.92378,0.92356,0.92374,170


In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [8]:
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, each_bar, n_bars, pip_movement, use_pullback):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None
    lookback = n_bars + 1 if use_pullback else n_bars
    lookforward = -1 if use_pullback else 0
    pip_movement /= divider

    for i in range(lookback, len(df)):
        curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]
        spread = abs(curr_ask_open - curr_bid_open)

        mid_opens = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Open'])
        mid_highs = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_High'])
        mid_lows = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Low'])
        mid_closes = list(df.loc[df.index[i - lookback:i + lookforward], 'Mid_Close'])

        if each_bar:
            buy_signal = all([mid_opens[j] < mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement for j in range(len(mid_opens))])
            sell_signal = all([mid_opens[j] > mid_closes[j] and abs(mid_opens[j] - mid_closes[j]) >= pip_movement for j in range(len(mid_opens))])

        else:
            buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
            sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

        if use_pullback and buy_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= 0.25 * abs(mid_high1 - mid_low1)

        if use_pullback and sell_signal:
            mid_open1, mid_high1, mid_low1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
            sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= 0.25 * abs(mid_high1 - mid_low1)

        highest_high, lowest_low = max(mid_highs), min(mid_lows)

        if trade is None:
            if buy_signal:
                open_price = float(curr_ask_open)
                stop_loss = round(lowest_low, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif sell_signal:
                open_price = float(curr_bid_open)
                stop_loss = round(highest_high, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)


        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [11]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.5, 2.0]
spread_cutoffs = [0.10, 0.15, 0.20]
each_bar_vals = [True, False]
n_bars_vals = [3, 5]
pip_movement_vals = [10, 15, 20, 25, 30, 35, 40]
use_pullback_vals = [True, False]

n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(each_bar_vals) * len(n_bars_vals) * len(pip_movement_vals) * len(use_pullback_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for each_bar in each_bar_vals:
            for n_bars in n_bars_vals:
                for pip_movement in pip_movement_vals:
                    for use_pullback in use_pullback_vals:
                        all_combos.append((risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_each_bar_val = None
best_n_bars_val = None
best_pip_movement_val = None
best_use_pullback_val = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, each_bar, n_bars, pip_movement, use_pullback)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'each_bar': each_bar, 'n_bars': n_bars, 'pip_movement': pip_movement, 'use_pullback': use_pullback})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_each_bar_val = each_bar
        best_n_bars_val = n_bars
        best_pip_movement_val = pip_movement
        best_use_pullback_val = use_pullback

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 336

-116.21038000000499
Num buys: 28
Num sells: 33
Num trades: 61
Num wins: 24
Num losses: 37
Win streak: 3
Loss streak: 6
Avg pips risked: 0.0027472131147541013
Remaining runs: 335
Best reward so far: -116.21038000000499

-47.03114000000036
Num buys: 0
Num sells: 1
Num trades: 1
Num wins: 0
Num losses: 1
Win streak: 0
Loss streak: 1
Avg pips risked: 0.006770000000000054
Remaining runs: 334
Best reward so far: -47.03114000000036

0
Num buys: 0
Num sells: 0
Num trades: 0
Num wins: 0
Num losses: 0
Win streak: 0
Loss streak: 0
Remaining runs: 333
Best reward so far: 0

-154.6144400000025
Num buys: 6
Num sells: 7
Num trades: 13
Num wins: 4
Num losses: 9
Win streak: 2
Loss streak: 4
Avg pips risked: 0.00387769230769233
Remaining runs: 332
Best reward so far: 0

0
Num buys: 0
Num sells: 0
Num trades: 0
Num wins: 0
Num losses: 0
Win streak: 0
Loss streak: 0
Remaining runs: 331
Best reward so far: 0

-144.3737000000022
Num buys: 4
Num sells: 5
Num trades: 9
Num wins: 2
Num losses: 7

In [12]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best each bar val: ' + str(best_each_bar_val))
print('Best n bars val: ' + str(best_n_bars_val))
print('Best pip movement val: ' + str(best_pip_movement_val))
print('Best use pullback val: ' + str(best_use_pullback_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 923.9451899999921
Best risk/reward ratio: 1.5
Best spread: 0.15
Best each bar val: False
Best n bars val: 3
Best pip movement val: 25
Best use pullback val: False
-----------------------
Top results:
{'reward': 807, 'ratio': 2.0, 'spread': 0.15, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 923, 'ratio': 1.5, 'spread': 0.15, 'each_bar': False, 'n_bars': 3, 'pip_movement': 25, 'use_pullback': False}
{'reward': 877, 'ratio': 1.5, 'spread': 0.2, 'each_bar': False, 'n_bars': 3, 'pip_movement': 15, 'use_pullback': True}
{'reward': 856, 'ratio': 1.5, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 15, 'use_pullback': True}
{'reward': 810, 'ratio': 2.0, 'spread': 0.2, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 805, 'ratio': 2.0, 'spread': 0.1, 'each_bar': False, 'n_bars': 3, 'pip_movement': 20, 'use_pullback': True}
{'reward': 920, 'ratio': 1.5, 